In [2]:
import pandas as pd

In [7]:
artists_df = pd.read_json('data/artists.jsonl')
artists_df

,id,name,genres
0,7jVv8c5Fj3E9VhNjxT4snq,Lil Nas X,"[country rap, lgbtq+ hip hop, pop rap, queer c..."
1,0xRXCcSX89eobfrshSVdyu,MEDUZA,"[dance pop, edm, pop dance, pop house, tropica..."
2,3OKg7YbOIatODzkRIbLJR4,TheFatRat,[speedrun]
3,4f7KfxeHq9BiylGmyXepGt,Tanishk Bagchi,"[desi pop, filmi, modern bollywood]"
4,56mfhUDKa1vec6rSLZV5Eg,Jawsh 685,[nz pop]
...,...,...,...
1662,6AMd49uBDJfhf30Ak2QR5s,Coi Leray,"[pop r&b, r&b, trap queen, viral rap]"
1663,6qgnBH6iDM91ipVXv28OMu,KAYTRANADA,"[escape room, indie soul, lgbtq+ hip hop]"
1664,5q8HGNo0BjLWaTAhRtbwxa,Brantley Gilbert,"[contemporary country, country, country road, ..."
1665,26T4yOaOoFJvUvxR87Y9HO,Bethel Music,"[ambient worship, ccm, christian music, deep c..."


In [3]:
sessions_df = pd.read_json('data/sessions.jsonl')
sessions_df

,session_id,timestamp,user_id,track_id,event_type
0,124,2022-07-07 10:10:12.000,101.0,6YpSiNQN8pVzJMOX2fXGHm,play
1,124,2022-07-07 10:11:57.833,101.0,6YpSiNQN8pVzJMOX2fXGHm,skip
2,124,2022-07-07 10:12:02.833,101.0,3oP2vxUR2bjFCrU9YolGnM,play
3,124,2022-07-07 10:13:29.385,101.0,None,advertisment
4,124,2022-07-07 10:13:29.385,101.0,3oP2vxUR2bjFCrU9YolGnM,like
...,...,...,...,...,...
56213,10123,2022-08-26 01:06:39.858,1100.0,2HVCLZ6dMDelXhloO5sAjV,like
56214,10123,2022-08-26 01:09:07.486,1100.0,0oTyoTKEKMdF3rXcOLyEfN,play
56215,10123,2022-08-26 01:12:29.086,1100.0,0oTyoTKEKMdF3rXcOLyEfN,skip
56216,10123,2022-08-26 01:12:34.086,1100.0,2N2s2s2w0jmSIPNZcm8Jnr,play


In [4]:
tracks_df = pd.read_json('data/tracks.jsonl')
tracks_df

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0RNxWy0PC3AyH4ThH3aGK6,Mack the Knife,55.0,201467,0,19eLuQmk9aCobbVDHc6eek,1929,0.673,0.3770,0,-14.141,0.0697,0.5860,0.000000,0.3320,0.7130,88.973
1,2W889aLIKxULEefrleFBFI,Someone to Watch Over Me,54.0,198000,0,1Mxqyy3pSjf8kZZL4QVxS0,1943,0.204,0.1510,2,-17.842,0.0418,0.9470,0.000009,0.3210,0.1340,91.783
2,4Pnzw1nLOpDNV6MKI5ueIR,Nancy (With the Laughing Face) - 78rpm Version,55.0,199000,0,1Mxqyy3pSjf8kZZL4QVxS0,1944,0.295,0.0826,1,-19.569,0.0367,0.9840,0.000358,0.1560,0.1690,128.600
3,7GLmfKOe5BfOXk7334DoKt,None,54.0,163000,0,1Mxqyy3pSjf8kZZL4QVxS0,1944,0.561,0.3350,9,-11.093,0.0499,0.8400,0.000002,0.7880,0.5900,126.974
4,6kD1SNGPkfX9LwaGd1FG92,None,53.0,186173,0,1Mxqyy3pSjf8kZZL4QVxS0,1944,0.197,0.0546,1,-22.411,0.0346,0.9500,0.276000,0.1520,0.1000,90.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,0LcNMuOiULmxJK3bdHTfDF,Missing You,65.0,214987,0,7gAppWoH7pcYmphCVTXkzs,2019-04-19,0.537,0.6470,2,-7.370,0.0519,0.1680,0.000000,0.1080,0.5310,79.770
22408,1uviKYHZuM4uINK33F7sCt,Fix It to Break It,70.0,198799,0,7okSU80WTrn4LXlyXYbX3P,2020-03-27,0.493,0.4610,2,-8.524,0.0456,0.8450,0.000000,0.1150,0.3500,51.414
22409,1fXmDeiCb3ABt5CzkMxp4u,Lotus Inn,63.0,195868,0,2jnIB6XdLvnJUeNTy5A0J2,2020-12-04,0.578,0.7400,2,-5.080,0.0721,0.0119,0.000000,0.1800,0.4950,155.984
22410,27kcZEJvhkb1rzZS9gCpdA,remember the mornings,67.0,202355,0,7okSU80WTrn4LXlyXYbX3P,2020-11-27,0.590,0.4430,0,-5.662,0.0722,0.6180,0.000000,0.1100,0.3420,149.447


In [5]:
users_df = pd.read_json('data/users.jsonl')
users_df

,user_id,name,city,street,favourite_genres,premium_user,id
0,101,Dominik Paździora,Wrocław,aleja Jabłoniowa 55/02,None,0.0,-1.0
1,102,Justyna Mszyca,Poznań,ul. Krótka 710,"[reggaeton, latin arena pop, modern rock]",0.0,NaN
2,103,Kornelia Witka,Gdynia,al. Kościuszki 587,"[rap, art rock, rock]",NaN,NaN
3,104,Jędrzej Kotala,Gdynia,pl. Kossaka 97/86,"[mexican pop, contemporary country, psychedeli...",0.0,NaN
4,105,Karol Kleban,Kraków,ul. Tulipanowa 861,"[rock, adult standards, permanent wave]",0.0,NaN
...,...,...,...,...,...,...,...
995,1096,Witold Podeszwa,Radom,pl. Ściegiennego 71/30,"[psychedelic rock, pop urbaine, alternative rock]",1.0,NaN
996,1097,Emil Brodawka,Szczecin,ulica Toruńska 71,None,1.0,NaN
997,1098,Anastazja Frydel,Gdynia,al. Plażowa 25,"[hard rock, alternative rock, regional mexican]",0.0,NaN
998,1099,Anastazja Szmurło,Szczecin,plac Przechodnia 13,"[modern rock, adult standards, electropop]",1.0,NaN
